In [41]:
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv 

load_dotenv()



True

In [42]:
class ChatState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [43]:
model = ChatGroq(model="llama-3.3-70b-versatile")


In [44]:
def gen_joke(state: ChatState):
    prompt = f"Tell me a funny joke about '{state['topic']}'."
    response = model.invoke(prompt).content
    return {"joke": response}

In [45]:
def gen_exp(state: ChatState):
    prompt = f"Explain the following joke in simple terms: '{state['joke']}'."
    response = model.invoke(prompt).content
    return {"explanation": response}

In [46]:
from langgraph.checkpoint.memory import InMemorySaver

checkpoint = InMemorySaver()

graph = StateGraph(ChatState)

graph.add_node("gen_joke", gen_joke)
graph.add_node("gen_exp", gen_exp)
graph.add_edge(START, "gen_joke")
graph.add_edge("gen_joke", "gen_exp")
graph.add_edge("gen_exp", END)

In [47]:
thread1 = 1
config = {"configurable":  { "thread_id": thread1}}

In [48]:
thread2 = 2
config2 = {"configurable":  { "thread_id": thread2}}

In [49]:
app = graph.compile(checkpointer=checkpoint)

In [50]:
first = {"topic": "computers"}
second= {"topic": "animals"}

In [51]:
result1 = app.invoke(first, config=config)
result2 = app.invoke(second, config=config2)

In [52]:
result1

{'topic': 'computers',
 'joke': 'Why do computers go to the doctor?\n\nBecause they have a virus! (get it?)',
 'explanation': 'This joke is funny because it\'s a play on words. \n\nIn medicine, a "virus" is something that makes people sick. But in computers, a "virus" is also a type of bad software that can harm the computer.\n\nSo, the joke is saying that computers go to the doctor because they have a "virus", but it\'s not the kind that people get. It\'s the kind that computers get. It\'s a clever and silly way to use the same word in two different ways to make a joke. The "(get it?)" at the end is the person telling the joke, asking if you understand the wordplay.'}

In [53]:
result2

{'topic': 'animals',
 'joke': 'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist.',
 'explanation': 'This joke is funny because it\'s a play on words. \n\n"Purr" is the sound that cats make, but in music, a "percussionist" is someone who plays drums or other instruments that you hit to make sound. \n\nSo, the joke is saying that the cat wants to be the "purr-cussionist" (like percussionist, but with "purr" instead) because it\'s a cat and it makes a "purr" sound. It\'s a silly and clever connection between the cat\'s sound and the music term.'}

In [54]:
history = list(app.get_state_history(config2))
for state in history:
    print(f"State: {state.values}")
    print(f"Next: {state.next}")
    print("---")

State: {'topic': 'animals', 'joke': 'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist.', 'explanation': 'This joke is funny because it\'s a play on words. \n\n"Purr" is the sound that cats make, but in music, a "percussionist" is someone who plays drums or other instruments that you hit to make sound. \n\nSo, the joke is saying that the cat wants to be the "purr-cussionist" (like percussionist, but with "purr" instead) because it\'s a cat and it makes a "purr" sound. It\'s a silly and clever connection between the cat\'s sound and the music term.'}
Next: ()
---
State: {'topic': 'animals', 'joke': 'Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist.'}
Next: ('gen_exp',)
---
State: {'topic': 'animals'}
Next: ('gen_joke',)
---
State: {}
Next: ('__start__',)
---
